In [ ]:
import mysql.connector
import time
import sys


# --- Function for READ UNCOMMITTED demo (dirty read) ---




def run_read_uncommitted():
   print("\n=== READ UNCOMMITTED Demo ===", flush=True)


   # Connection 1
   conn1 = mysql.connector.connect(
       host="localhost", user="root", password="<password>",
       database="FINAL_URP", autocommit=False
   )
   cursor1 = conn1.cursor()
   cursor1.execute(
       "SET SESSION TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;")


   # Connection 2
   conn2 = mysql.connector.connect(
       host="localhost", user="root", password="<password>",
       database="FINAL_URP", autocommit=False
   )
   cursor2 = conn2.cursor()
   cursor2.execute(
       "SET SESSION TRANSACTION ISOLATION LEVEL READ UNCOMMITTED;")


   try:
       # --- Step 0: Reset score to a known value ---
       cursor1.execute("UPDATE Review SET score=3 WHERE review_id=1;")
       conn1.commit()
       print("Score reset to 3 for a predictable demo", flush=True)


       # --- Transaction 1: temporary update ---
       cursor1.execute("SELECT score FROM Review WHERE review_id=1;")
       original_score = cursor1.fetchone()[0]
       temp_score = original_score + 1  # safe increment
       cursor1.execute(
           "UPDATE Review SET score=%s WHERE review_id=1;", (temp_score,))
       print(
           f"Transaction 1 updates score to {temp_score} (uncommitted)", flush=True)


       # Simulate concurrency
       time.sleep(1)


       # --- Transaction 2 reads the dirty value ---
       cursor2.execute("SELECT score FROM Review WHERE review_id=1;")
       print("Transaction 2 reads score (dirty read):",
             cursor2.fetchone()[0], flush=True)


       # --- Rollback Transaction 1 ---
       conn1.rollback()
       cursor1.execute("SELECT score FROM Review WHERE review_id=1;")
       print("After rollback, Transaction 1 score:",
             cursor1.fetchone()[0], flush=True)


       # Optional: permanent update to show commit
       cursor1.execute("UPDATE Review SET score=5 WHERE review_id=1;")
       conn1.commit()
       cursor1.execute("SELECT score FROM Review WHERE review_id=1;")
       print("After commit, Transaction 1 score:",
             cursor1.fetchone()[0], flush=True)


   except mysql.connector.Error as err:
       conn1.rollback()
       conn2.rollback()
       print("Error:", err, flush=True)


   finally:
       cursor1.close()
       conn1.close()
       cursor2.close()
       conn2.close()




# --- Run the demo ---
if __name__ == "__main__":
   run_read_uncommitted()


# READ COMMITTED


# --- Function for READ COMMITTED demo ---




def run_read_committed():
   print("\n=== READ COMMITTED Demo ===", flush=True)


   # Connection 1
   conn1 = mysql.connector.connect(
       host="localhost", user="root", password="<password>
       database="FINAL_URP", autocommit=False
   )
   cursor1 = conn1.cursor()
   cursor1.execute("SET SESSION TRANSACTION ISOLATION LEVEL READ COMMITTED;")


   # Connection 2
   conn2 = mysql.connector.connect(
       host="localhost", user="root", password="<password>",
       database="FINAL_URP", autocommit=False
   )
   cursor2 = conn2.cursor()
   cursor2.execute("SET SESSION TRANSACTION ISOLATION LEVEL READ COMMITTED;")


   try:
       # --- Step 0: Reset score to a known value ---
       cursor1.execute("UPDATE Review SET score=3 WHERE review_id=1;")
       conn1.commit()
       print("Score reset to 3 for a predictable demo", flush=True)


       # --- Transaction 1: temporary update ---
       cursor1.execute("SELECT score FROM Review WHERE review_id=1;")
       original_score = cursor1.fetchone()[0]
       temp_score = original_score + 1  # safe increment
       cursor1.execute(
           "UPDATE Review SET score=%s WHERE review_id=1;", (temp_score,))
       print(
           f"Transaction 1 updates score to {temp_score} (uncommitted)", flush=True)


       time.sleep(1)  # simulate concurrency


       # --- Transaction 2 reads: cannot see uncommitted changes ---
       cursor2.execute("SELECT score FROM Review WHERE review_id=1;")
       print("Transaction 2 reads score (no dirty read):",
             cursor2.fetchone()[0], flush=True)


       # --- Rollback Transaction 1 ---
       conn1.rollback()
       cursor1.execute("SELECT score FROM Review WHERE review_id=1;")
       print("After rollback, Transaction 1 score:",
             cursor1.fetchone()[0], flush=True)


       # --- Permanent update to show commit ---
       cursor1.execute("UPDATE Review SET score=5 WHERE review_id=1;")
       conn1.commit()
       cursor1.execute("SELECT score FROM Review WHERE review_id=1;")
       print("After commit, Transaction 1 score:",
             cursor1.fetchone()[0], flush=True)


   except mysql.connector.Error as err:
       conn1.rollback()
       conn2.rollback()
       print("Error:", err, flush=True)


   finally:
       cursor1.close()
       conn1.close()
       cursor2.close()
       conn2.close()




# --- Run the demo ---
if __name__ == "__main__":
   run_read_committed()


#  REPEATED READ


# --- Function for REPEATABLE READ demo ---




def run_repeatable_read():
   print("\n=== REPEATABLE READ Demo ===", flush=True)


   # Connection 1
   conn1 = mysql.connector.connect(
       host="localhost", user="root", password="<password>",
       database="FINAL_URP", autocommit=False
   )
   cursor1 = conn1.cursor()
   cursor1.execute("SET SESSION TRANSACTION ISOLATION LEVEL REPEATABLE READ;")


   # Connection 2
   conn2 = mysql.connector.connect(
       host="localhost", user="root", password="<password>",
       database="FINAL_URP", autocommit=False
   )
   cursor2 = conn2.cursor()
   cursor2.execute("SET SESSION TRANSACTION ISOLATION LEVEL REPEATABLE READ;")


   try:
       # --- Step 0: Reset score to a known value ---
       cursor1.execute("UPDATE Review SET score=3 WHERE review_id=1;")
       conn1.commit()
       print("Score reset to 3 for a predictable demo", flush=True)


       # --- Transaction 1: first read ---
       cursor1.execute("SELECT score FROM Review WHERE review_id=1;")
       first_read = cursor1.fetchone()[0]
       print("Transaction 1 reads score first time:", first_read, flush=True)


       # --- Transaction 1: temporary update (will rollback) ---
       temp_score = first_read + 1  # safe increment
       cursor1.execute(
           "UPDATE Review SET score=%s WHERE review_id=1;", (temp_score,))
       print(
           f"Transaction 1 updates score to {temp_score} (temporary, uncommitted)", flush=True)


       # Transaction 2 reads during Transaction 1 (should still see old value)
       cursor2.execute("SELECT score FROM Review WHERE review_id=1;")
       print("Transaction 2 reads score (before rollback):",
             cursor2.fetchone()[0], flush=True)


       # --- Rollback Transaction 1 ---
       conn1.rollback()
       cursor1.execute("SELECT score FROM Review WHERE review_id=1;")
       print("After rollback, Transaction 1 score:",
             cursor1.fetchone()[0], flush=True)


       # --- Permanent update to show commit ---
       cursor1.execute("UPDATE Review SET score=5 WHERE review_id=1;")
       conn1.commit()
       cursor1.execute("SELECT score FROM Review WHERE review_id=1;")
       print("After commit, Transaction 1 score:",
             cursor1.fetchone()[0], flush=True)


   except mysql.connector.Error as err:
       conn1.rollback()
       conn2.rollback()
       print("Error:", err, flush=True)


   finally:
       cursor1.close()
       conn1.close()
       cursor2.close()
       conn2.close()




# --- Run the demo ---
if __name__ == "__main__":
   run_repeatable_read()




#  SERIALIZABLE




def run_serializable():
   print("\n=== SERIALIZABLE Demo ===", flush=True)


   # Single connection demo for rollback/commit
   conn = mysql.connector.connect(
       host="localhost", user="root", password="<password>",
       database="FINAL_URP", autocommit=False
   )
   cursor = conn.cursor()
   cursor.execute("SET SESSION TRANSACTION ISOLATION LEVEL SERIALIZABLE;")


   try:
       # Reset score
       cursor.execute("UPDATE Review SET score=3 WHERE review_id=1;")
       conn.commit()
       print("Score reset to 3", flush=True)


       # Temporary update
       cursor.execute("SELECT score FROM Review WHERE review_id=1;")
       original_score = cursor.fetchone()[0]
       temp_score = original_score + 1
       cursor.execute(
           "UPDATE Review SET score=%s WHERE review_id=1;", (temp_score,))
       print(f"Temporary update to {temp_score} (uncommitted)", flush=True)


       # Rollback
       conn.rollback()
       cursor.execute("SELECT score FROM Review WHERE review_id=1;")
       print("After rollback:", cursor.fetchone()[0], flush=True)


       # Permanent update
       cursor.execute("UPDATE Review SET score=5 WHERE review_id=1;")
       conn.commit()
       cursor.execute("SELECT score FROM Review WHERE review_id=1;")
       print("After commit:", cursor.fetchone()[0], flush=True)


   except mysql.connector.Error as err:
       conn.rollback()
       print("Error:", err, flush=True)


   finally:
       cursor.close()
       conn.close()




if __name__ == "__main__":
   run_serializable()
